In [ ]:
# default_exp database

# Database Class

> API details.

In [ ]:
#hide
import logging
logging.basicConfig(level= logging.WARNING)
log = logging.getLogger("pynamodb")
log.setLevel(logging.DEBUG)
log.setLevel(logging.WARNING)
log.propagate = True


In [ ]:
#hide
import pickle, os, json

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
os.environ['INTCOLS'] = json.dumps(['cprcode', 'iprcode', 'oprcode', 'pr_barcode', 'pr_barcode2', 'sellingPrice'])
REGION = 'ap-southeast-1'

In [ ]:
#hide
print(json.dumps(os.environ['INTCOLS']))

"[\"cprcode\", \"iprcode\", \"oprcode\", \"pr_barcode\", \"pr_barcode2\", \"sellingPrice\"]"


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
from datetime import datetime
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BooleanAttribute, BinaryAttribute
from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
from botocore.config import Config
from s3bz.s3bz import S3
from pprint import pprint
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict
from nicHelper.exception import errorString
from nicHelper import pdUtils
from awsSchema.apigateway import Response, Event
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses import dataclass
from typing import List
from villaProductDatabase.query import Querier
from villaProductDatabase.helpers import Helpers
from villaProductDatabase.s3 import S3Cache
from villaProductDatabase.schema import KeySchema, createIndex
from villaProductDatabase.update import Updater
from requests import post

import pickle, json, boto3, bz2, requests, validators, os, logging, traceback, zlib

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
import os

DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
DBHASHLOCATION = '/mnt/efs/database.hash'
DBCACHELOCATION = '/mnt/efs/database.cache'
INTCOLS = json.loads(os.environ.get('INTCOLS') or '{}')
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
except KeyError as e:
  DAX_ENDPOINT = None
  print(f'dax endpoint missing {e}')
  
print(DAX_ENDPOINT)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


# Main Database Class

In [ ]:
#export
# dont forget to import dependent classes from the relevant notebooks
class ProductDatabase( Querier, Helpers, KeySchema, S3Cache, Updater):
  class Meta:
    aws_access_key_id = ACCESS_KEY_ID
    aws_secret_access_key = SECRET_ACCESS_KEY
    table_name = DATABASE_TABLE_NAME
    region = REGION
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None


  TRUE = 'Y'
  FALSE = 'N'
  
  # indexes
  needsUpdateIndex = createIndex('needsUpdate','sellingPrice')
  cprcodeIndex = createIndex('cprcode', 'sellingPrice')
  oprcodeIndex = createIndex('oprcode', 'sellingPrice')
  pr_dpcodeIndex = createIndex('pr_dpcode', 'sellingPrice')
  pr_barcodeIndex = createIndex('pr_barcode', 'sellingPrice')
  pr_barcode2Index = createIndex('pr_barcode2', 'sellingPrice')
  pr_suref3Index = createIndex('pr_suref3', 'sellingPrice')
  pr_sa_methodIndex = createIndex('pr_sa_method', 'sellingPrice')
  
  
    
  def __repr__(self):
    return self.returnKW(self.data)
  
    
  @staticmethod
  def returnKW(inputDict):
    outputStr = 'ProductDatabase Object\n'
    for k,v in inputDict.items():
      outputStr += f'{k} {v}\n'
    return outputStr  
  @staticmethod
  def convertIntCols(products:List[dict])->List[dict]:
    convertedProducts = [{k:int(float(v)) if k in INTCOLS else v for k,v in product.items()}for product in sampleProducts]
    return convertedProducts
    
  
    

In [ ]:
%%time
#hide
## set items to udpate
with ProductDatabase.batch_write() as batch:
  for product in ProductDatabase.scan(limit=500):
    product.setUpdate(batch=batch)

CPU times: user 302 ms, sys: 24.6 ms, total: 326 ms
Wall time: 844 ms


In [ ]:
#hide
##tests
assert ProductDatabase.Meta.table_name == 'product-table-dev-manual', 'product table name not set'
assert len(list(ProductDatabase.scan(limit=1))) == 1, 'db scan not working'
assert len(list(ProductDatabase.needsUpdateIndex.query(ProductDatabase.TRUE))) > 1, 'cant find any needsUpdate item'

In [ ]:
#hide
### set test env
sampleProducts = [{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0235141', 'iprcode': '0235141', 'oprcode': '0235141', 'ordertype': 'Y', 'pr_abb': 'EEBOO-PZCT3-PUZZLE', 'pr_active': 'Y', 'pr_cgcode': '08', 'pr_code': '0235141', 'pr_dpcode': '19', 'pr_engname': 'EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3', 'pr_ggcode': '113', 'pr_market': 'eeboo,PUZZLE-PZCT3', 'pr_name': 'EEBOO-PZCT3-ตัวต่อนับเลข ANIMAL COUNTING_3ED', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM1979    ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0217153', 'iprcode': '0217153', 'oprcode': '0217153', 'ordertype': 'Y', 'pr_abb': 'COCOA LOCO MILK CHOC', 'pr_active': 'Y', 'pr_cgcode': '98', 'pr_code': '0217153', 'pr_dpcode': '28', 'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_ggcode': '003', 'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL', 'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_puqty': '24', 'pr_sa_method': '1', 'pr_sucode1': 'F1222     ', 'pr_suref3': 'S', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0182223', 'iprcode': '0182223', 'oprcode': '0182223', 'ordertype': 'Y', 'pr_abb': 'CIRIO PIZZASSIMO 400', 'pr_active': 'Y', 'pr_cgcode': '06', 'pr_code': '0182223', 'pr_dpcode': '06', 'pr_engname': 'CIRIO PIZZASSIMO 400G.', 'pr_ggcode': '004', 'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '2589      ', 'pr_suref3': 'C', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0124461', 'iprcode': '0124461', 'oprcode': '0124461', 'ordertype': 'Y', 'pr_abb': 'NEW CHOICE LYCHEE', 'pr_active': 'Y', 'pr_cgcode': '02', 'pr_code': '0124461', 'pr_dpcode': '02', 'pr_engname': 'NEW CHOICE LYCHEE', 'pr_ggcode': '003', 'pr_market': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_name': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '695       ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]

### Update with dictionary

In [ ]:
%%time
## get original item
item = next(ProductDatabase.query(9))
print(f'engname is {item.data.get("pr_engname")}')

### update item
updatedItem = ProductDatabase.updateWithDict(item,{'pr_engname':'testName'})
assert updatedItem.data.get('pr_engname') == 'testName'
print(f' name is changed to  {updatedItem.data.get("pr_engname")}')

engname is JOHNNIE WALKER RED 70 CL.
 name is changed to  testName
CPU times: user 0 ns, sys: 3.61 ms, total: 3.61 ms
Wall time: 6.57 ms


## Batch load and save data

In [ ]:
%%time
data = ProductDatabase.loadFromCache()
assert data.shape[0] > 10000, 'loadFrom cache not working'

loading hashkey allData-hash
loaded hash is be55bb1dccdf79efbf5b20255abf91b0cdf154c3
data is still in sync, using local cache
CPU times: user 314 ms, sys: 91.5 ms, total: 405 ms
Wall time: 593 ms


### Dump database to s3

In [ ]:
# %%time
# ProductDatabase.resetS3Cache()

In [ ]:
#export
@add_class_method(ProductDatabase)
def cacheDb(cls, bucketName = INVENTORY_BUCKET_NAME, key = 'allData', limit=100, **kwargs):
  '''cache db to s3 and local efs'''
  db:pd.DataFrame = cls.loadFromCache(key=key, localCache=DBCACHELOCATION,
                                      localHash=DBHASHLOCATION, bucket=bucketName)
  logging.debug(f'origin item is shape{db.shape}')
  #### get change
  print('quering for changes')
  changes = list(cls.needsUpdateIndex.query(cls.TRUE, limit=limit))
  print('convert to df')
  changesDf = cls.toDf(changes)
  changesDf.set_index('cprcode', inplace=True)
  print(changesDf.shape)
#   db.set_index('cprcode', inplace=True)
  updatedDb = db.append(changesDf).drop_duplicates('cprcode', keep='last')
#   updatedDb.reset_index(inplace=True)
  cls.saveRemoteCache(updatedDb)
  print('saving to remote cache')
  with cls.batch_write() as batch:
    for item in changes:
      item.setNoUpdate(batch=batch)
  cls.notify(f'db shape is {db.shape}')
  return updatedDb

In [ ]:
%%time
# originalDb = ProductDatabase.loadFromCache()
# listPd = list(product.needsUpdateIndex.query('Y', limit=5))
# changesDf = ProductDatabase.toDf(listPd)
# changesDf.set_index('cprcode', inplace=True)
# originalDb.set_index('cprcode',inplace=True)
# newDf = originalDb.append(changesDf)
# newDf.reset_index(inplace=True)
# originalDb

ProductDatabase.cacheDb()

loading hashkey allData-hash
loaded hash is be55bb1dccdf79efbf5b20255abf91b0cdf154c3
data is still in sync, using local cache
quering for changes
convert to df
(100, 49)
hashKey is allData-hash
saving hash to s3
saved hash 1515594397ef54de982d91763edb374f035cfca0
saving to remote cache
CPU times: user 5 s, sys: 493 ms, total: 5.49 s
Wall time: 7.55 s


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,hema_sizedesc,pr_brand_en,pr_brand_th,pr_online_name_en,pr_online_name_th,hema_name_en,hema_name_th,pr_name_en,pr_name_th,sort_weight
0.0,225407.0,225407.0,225407.0,Y,MOKU,Y,87,225407,21.0,KONJAC LINGUINI,...,none,none,none,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,0.0
1.0,241101.0,241101.0,241101.0,Y,BUMILGOCHUJANG,Y,06,0241101,8.0,BUMILGOCHUJANG,...,none,none,none,BUMILGOCHUJANG,BUMILGOCHUJANG,none,none,BUMILGOCHUJANG,บูมิลโคชูจังซอสเผ้ดเกาหลี,0.0
2.0,190100.0,190100.0,190100.0,Y,WHITE SHIMEJI,Y,5,190100,19.0,WHITE SHIMEJI,...,none,none,none,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,WHITE SHIMEJI,QUALITY VEGETABLE เห็ดชิเมจิขาว,0.0
3.0,62644.0,62644.0,62644.0,Y,STUTE APRICOT JAM430,Y,7,62644,8.0,STUTE APRICOT JAM 430 G.,...,430g,STUTE,STUTE,Stute Apricot Jam 430g,Stute Apricot Jam 430g,Apricot Jam,Apricot Jam,STUTE APRICOT JAM 430 G.,STUTE APRICOT JAM 430 G.,0.0
4.0,192167.0,192167.0,192167.0,Y,**COLGATE TOTAL CHAR,Y,10,192167,9.0,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,...,150g,COLGATE,COLGATE,Colgate Total Charcoal Deep Clean 150g,Colgate Total Charcoal Deep Clean 150g,Total Charcoal Deep Clean,Total Charcoal Deep Clean,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,COGATE ยสฟ.โททอลชาร์โคลดีพคลีน 150กรัม,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43798.0,236128.0,236128.0,236128.0,Y,CRANBERRY CIABATTA 8,Y,15,0236128,19.0,CRANBERRY CIABATTA 85G,...,none,none,none,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,แครนเบอร์รี่เชียบัตต้า 85G,0.0
43799.0,240795.0,240795.0,240795.0,Y,Nature’s Best,Y,06,0240795,2.0,NATURE’S BEST CHESTNUTS WITH SHELL,...,none,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,NATURE’S BEST CHESTNUTS WITH SHELL,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,เนเจอร์ส เบสท์ เมล็ดเกาลัดคั่วพร้อมเปลือก,0.0
43800.0,178950.0,178950.0,178950.0,Y,TIPBAKERY 25 ฿,Y,15,178950,19.0,TIPBAKERY 25 ฿,...,none,none,none,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,ทิพย์เบเกอรี่ 25 ฿,0.0
43801.0,238167.0,238167.0,238167.0,Y,HUGOS CHOCOLATE ROAS,Y,04,0238167,2.0,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,...,none,none,none,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS ช็อกโกแลตเคลือบเมล็ดอัลมอนด์ 1121 120G.,0.0


In [ ]:
@add_class_method(ProductDatabase)
def dumpToS3(cls, bucketName= INVENTORY_BUCKET_NAME, key = 'allData', limit=500, **kwargs):
  return cls.cacheDb(limit=1000)


In [ ]:
%%time
ProductDatabase.dumpToS3()

loading hashkey allData-hash
loaded hash is 1515594397ef54de982d91763edb374f035cfca0
data is still in sync, using local cache
quering for changes
convert to df
(1000, 49)
hashKey is allData-hash
saving hash to s3
saved hash 8f9240e80783f1ada52376d3afde843ae02dbf1a
saving to remote cache
CPU times: user 20.5 s, sys: 536 ms, total: 21 s
Wall time: 23.7 s


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,hema_sizedesc,pr_brand_en,pr_brand_th,pr_online_name_en,pr_online_name_th,hema_name_en,hema_name_th,pr_name_en,pr_name_th,sort_weight
0.0,225407.0,225407.0,225407.0,Y,MOKU,Y,87,225407,21.0,KONJAC LINGUINI,...,none,none,none,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,0.0
1.0,241101.0,241101.0,241101.0,Y,BUMILGOCHUJANG,Y,06,0241101,8.0,BUMILGOCHUJANG,...,none,none,none,BUMILGOCHUJANG,BUMILGOCHUJANG,none,none,BUMILGOCHUJANG,บูมิลโคชูจังซอสเผ้ดเกาหลี,0.0
2.0,190100.0,190100.0,190100.0,Y,WHITE SHIMEJI,Y,5,190100,19.0,WHITE SHIMEJI,...,none,none,none,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,WHITE SHIMEJI,QUALITY VEGETABLE เห็ดชิเมจิขาว,0.0
3.0,62644.0,62644.0,62644.0,Y,STUTE APRICOT JAM430,Y,7,62644,8.0,STUTE APRICOT JAM 430 G.,...,430g,STUTE,STUTE,Stute Apricot Jam 430g,Stute Apricot Jam 430g,Apricot Jam,Apricot Jam,STUTE APRICOT JAM 430 G.,STUTE APRICOT JAM 430 G.,0.0
4.0,192167.0,192167.0,192167.0,Y,**COLGATE TOTAL CHAR,Y,10,192167,9.0,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,...,150g,COLGATE,COLGATE,Colgate Total Charcoal Deep Clean 150g,Colgate Total Charcoal Deep Clean 150g,Total Charcoal Deep Clean,Total Charcoal Deep Clean,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,COGATE ยสฟ.โททอลชาร์โคลดีพคลีน 150กรัม,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43798.0,236128.0,236128.0,236128.0,Y,CRANBERRY CIABATTA 8,Y,15,0236128,19.0,CRANBERRY CIABATTA 85G,...,none,none,none,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,แครนเบอร์รี่เชียบัตต้า 85G,0.0
43799.0,240795.0,240795.0,240795.0,Y,Nature’s Best,Y,06,0240795,2.0,NATURE’S BEST CHESTNUTS WITH SHELL,...,none,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,NATURE’S BEST CHESTNUTS WITH SHELL,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,เนเจอร์ส เบสท์ เมล็ดเกาลัดคั่วพร้อมเปลือก,0.0
43800.0,178950.0,178950.0,178950.0,Y,TIPBAKERY 25 ฿,Y,15,178950,19.0,TIPBAKERY 25 ฿,...,none,none,none,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,ทิพย์เบเกอรี่ 25 ฿,0.0
43801.0,238167.0,238167.0,238167.0,Y,HUGOS CHOCOLATE ROAS,Y,04,0238167,2.0,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,...,none,none,none,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS ช็อกโกแลตเคลือบเมล็ดอัลมอนด์ 1121 120G.,0.0


In [ ]:
#export
def lambdaDumpToS3(event, _):
  result = ProductDatabase.cacheDb(limit = 500)
  return Response.getReturn(body = {'result': result.iloc[0].to_dict()})

In [ ]:
%%time
result = Response.fromDict(lambdaDumpToS3('','')).body['result']
print(result)

loading hashkey allData-hash
loaded hash is 8f9240e80783f1ada52376d3afde843ae02dbf1a
data is still in sync, using local cache
quering for changes
convert to df
(500, 49)
hashKey is allData-hash
saving hash to s3
saved hash c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
saving to remote cache
{'cprcode': 225407.0, 'iprcode': 225407.0, 'oprcode': 225407.0, 'ordertype': 'Y', 'pr_abb': 'MOKU', 'pr_active': 'Y', 'pr_cgcode': '87', 'pr_code': '225407', 'pr_dpcode': '21.0', 'pr_engname': 'KONJAC LINGUINI', 'pr_ggcode': '10', 'pr_market': 'MOKU', 'pr_name': 'บุกเส้นแบน ตราโมกุ', 'pr_puqty': '1.0', 'pr_sa_method': '1', 'pr_sucode1': 'F1148', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1.0', 'pstype': '1.0', 'pr_country_th': 'none', 'pr_country_en': 'Thailand', 'pr_keyword_th': 'none', 'pr_keyword_en': 'none', 'pr_filter_th': 'none', 'pr_filter_en': 'none', 'online_category_l1_th': 'อาหารสำเร็จรูปและไข่', 'online_category_l1_en': 'Delicatessen & Eggs', 'online_category_l2_th': 'อาหารพร้อมทาน', 'online_

In [ ]:
product.loadFromCache()

loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,...,hema_sizedesc,pr_brand_en,pr_brand_th,pr_online_name_en,pr_online_name_th,hema_name_en,hema_name_th,pr_name_en,pr_name_th,sort_weight
0,225407.0,225407.0,225407.0,Y,MOKU,Y,87,225407,21.0,KONJAC LINGUINI,...,none,none,none,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,KONJAC LINGUINI,บุกเส้นแบน ตราโมกุ,0.0
1,241101.0,241101.0,241101.0,Y,BUMILGOCHUJANG,Y,06,0241101,8.0,BUMILGOCHUJANG,...,none,none,none,BUMILGOCHUJANG,BUMILGOCHUJANG,none,none,BUMILGOCHUJANG,บูมิลโคชูจังซอสเผ้ดเกาหลี,0.0
2,190100.0,190100.0,190100.0,Y,WHITE SHIMEJI,Y,5,190100,19.0,WHITE SHIMEJI,...,none,none,none,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,White Shimeji,QUALITY VEGETABLE เห็ดชิเมจิขาว,WHITE SHIMEJI,QUALITY VEGETABLE เห็ดชิเมจิขาว,0.0
3,62644.0,62644.0,62644.0,Y,STUTE APRICOT JAM430,Y,7,62644,8.0,STUTE APRICOT JAM 430 G.,...,430g,STUTE,STUTE,Stute Apricot Jam 430g,Stute Apricot Jam 430g,Apricot Jam,Apricot Jam,STUTE APRICOT JAM 430 G.,STUTE APRICOT JAM 430 G.,0.0
4,192167.0,192167.0,192167.0,Y,**COLGATE TOTAL CHAR,Y,10,192167,9.0,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,...,150g,COLGATE,COLGATE,Colgate Total Charcoal Deep Clean 150g,Colgate Total Charcoal Deep Clean 150g,Total Charcoal Deep Clean,Total Charcoal Deep Clean,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,COGATE ยสฟ.โททอลชาร์โคลดีพคลีน 150กรัม,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43798,236128.0,236128.0,236128.0,Y,CRANBERRY CIABATTA 8,Y,15,0236128,19.0,CRANBERRY CIABATTA 85G,...,none,none,none,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,CRANBERRY CIABATTA 85G,แครนเบอร์รี่เชียบัตต้า 85G,0.0
43799,240795.0,240795.0,240795.0,Y,Nature’s Best,Y,06,0240795,2.0,NATURE’S BEST CHESTNUTS WITH SHELL,...,none,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,NATURE’S BEST CHESTNUTS WITH SHELL,none,none,NATURE’S BEST CHESTNUTS WITH SHELL,เนเจอร์ส เบสท์ เมล็ดเกาลัดคั่วพร้อมเปลือก,0.0
43800,178950.0,178950.0,178950.0,Y,TIPBAKERY 25 ฿,Y,15,178950,19.0,TIPBAKERY 25 ฿,...,none,none,none,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,TIPBAKERY 25 ฿,ทิพย์เบเกอรี่ 25 ฿,0.0
43801,238167.0,238167.0,238167.0,Y,HUGOS CHOCOLATE ROAS,Y,04,0238167,2.0,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,...,none,none,none,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS CHOCOLATE ROASTED ALMONDS 120G.,HUGOS ช็อกโกแลตเคลือบเมล็ดอัลมอนด์ 1121 120G.,0.0


## Save using Standard (instant save)

In [ ]:
#export
@dataclass_json(undefined=Undefined.INCLUDE)
@dataclass
class Product:
  iprcode: int
  cprcode: int
  data: CatchAll
@dataclass_json
@dataclass
class ValueUpdate:
  items: List[Product]

In [ ]:
#export
def chunks(l, n): return [l[x: x+n] for x in range(0, len(l), n)]

In [ ]:
chunks(list(range(100)),10)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
 [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
 [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]

In [ ]:
sampleProducts = [{ 'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] #ProductDatabase.valueUpdate({'items':sampleProducts})
ProductDatabase.valueUpdate2({'items':ProductDatabase.convertIntCols(sampleProducts)})

products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
time taken for validation  -0.38146 ms
divided into chunks -0.382286 ms
loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
get all from s3 -1.091853 ms


{'success': 1,
 'failure': 0,
 'skipped': 0,
 'failureMessage': [],
 'timetaken(ms)': 1163.942}

In [ ]:
#export
def lambdaUpdateProduct (event, _):
  products = Event.parseBody(event)['products']
  convertedProducts = ProductDatabase.convertIntCols(products)
  result = ProductDatabase.valueUpdate2({'items':convertedProducts})
  return Response.getReturn(body = result)

In [ ]:
event = Event.getInput(body = {'products':sampleProducts})
lambdaUpdateProduct(event, '')

products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
time taken for validation  -0.020947 ms
divided into chunks -0.021035 ms
loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
get all from s3 -0.531852 ms


{'body': '{"success":1,"failure":0,"skipped":0,"failureMessage":[],"timetaken(ms)":598.636}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

## Save using s3

### update using s3 link

In [ ]:
#export
@add_class_method(ProductDatabase)
def updateS3Input(cls, inputBucketName:str = INPUT_BUCKET_NAME, key:str = '', **kwargs):
  products = S3.load(key=key, bucket = inputBucketName,  **kwargs)
  updateResult = cls.valueUpdate2({'items':cls.convertIntCols(products)})
  return updateResult


In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = sampleProducts , 
                     bucket = INPUT_BUCKET_NAME,
                     accelerate = True)
logging.info('test input data saved to s3')
updateResult = ProductDatabase.updateS3Input( inputBucketName=INPUT_BUCKET_NAME, key= inputKeyName)

logging.info(f's3 save result is {saveResult} update result is {updateResult}')

products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
time taken for validation  -0.024872 ms
divided into chunks -0.024968 ms
loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
get all from s3 -0.819929 ms


In [ ]:
#export
def lambdaUpdateS3(event, _):
  inputKeyName = Event.from_dict(event).key()
  try:
    updateResult = ProductDatabase.updateS3Input(
      inputBucketName=INPUT_BUCKET_NAME, key= inputKeyName)
  except:
    ProductDatabase.notify(f'error updating with s3 {errorString()}')
    return Response.returnError(errorString())
  
  
  
  ProductDatabase.notify(f'success update {updateResult}')
  return Response.getReturn(body = updateResult)

In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = sampleProducts , 
                     bucket = INPUT_BUCKET_NAME)
event = Event.getInput({'key': inputKeyName})
lambdaUpdateS3(event, '')

products are [{'cprcode': 171670, 'iprcode': 171670, 'oprcode': 171670, 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]
time taken for validation  -0.021385 ms
divided into chunks -0.021568 ms
loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
get all from s3 -0.575215 ms


{'body': '{"success":1,"failure":0,"skipped":0,"failureMessage":[],"timetaken(ms)":642.068}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

## Query test

### Product Query

In [ ]:
#export
@dataclass_json
@dataclass
class ProductsFromList:
  iprcodes: List[str]

In [ ]:
#export
def lambdaProductsFromList(event, *args):
  productsFromList = Event.parseDataClass(ProductsFromList,event)
  result:pd.DataFrame = ProductDatabase.productsFromList(productsFromList.iprcodes)
  results:List[ProductDatabase] = ProductDatabase.fromDf(result)
  resultDicts:List[dict] = ProductDatabase.toListDict(results)
  return Response.returnSuccess(resultDicts)

In [ ]:
%%time
event = Event.getInput({'iprcodes': ['0217153','203915','0000009']})
response = lambdaProductsFromList(event)
result = Response.parseBody(response)
assert len(result) == 3, f'result length is not 3 {len(result)}'
assert type(result[0]) == dict, 'result type is not dict'

loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
db shape is (43803, 50)
CPU times: user 287 ms, sys: 56.9 ms, total: 344 ms
Wall time: 465 ms


In [ ]:
event = Event.getInput({'iprcodes':['0000009','203915','0217153']})
lambdaProductsFromList(event)

loading hashkey allData-hash
loaded hash is c9ffa27c2e8e501717abcd9c58f2d5fb902d847d
data is still in sync, using local cache
db shape is (43803, 50)


{'body': '[{"cprcode":203915.0,"iprcode":203915.0,"oprcode":203915.0,"ordertype":"Y","pr_abb":"MEIJI GUMMY CHOCO GR","pr_active":"Y","pr_cgcode":"4","pr_code":"203915","pr_dpcode":"2.0","pr_engname":"MEIJI GUMMY CHOCO GREEN GRAPE 30G","pr_ggcode":"2","pr_market":"\\u0e40\\u0e21\\u0e08\\u0e34 \\u0e01\\u0e31\\u0e21\\u0e21\\u0e35\\u0e48 \\u0e0a\\u0e47\\u0e2d\\u0e01\\u0e42\\u0e01 \\u0e01\\u0e23\\u0e35\\u0e19\\u0e40\\u0e01\\u0e23\\u0e1b 30","pr_name":"\\u0e40\\u0e21\\u0e08\\u0e34 \\u0e01\\u0e31\\u0e21\\u0e21\\u0e35\\u0e48 \\u0e0a\\u0e47\\u0e2d\\u0e01\\u0e42\\u0e01 \\u0e01\\u0e23\\u0e35\\u0e19\\u0e40\\u0e01\\u0e23\\u0e1b 30 \\u0e01\\u0e23\\u0e31\\u0e21","pr_puqty":"20.0","pr_sa_method":"1","pr_sucode1":"2943","pr_suref3":"C","prtype":"I","psqty":"1.0","pstype":"1.0","pr_country_th":"none","pr_country_en":"China","pr_keyword_th":"none","pr_keyword_en":"none","pr_filter_th":"Grocery,Chocolate","pr_filter_en":"Grocery,Chocolate","online_category_l1_th":"\\u0e2a\\u0e34\\u0e19\\u0e04\\u0e49\\u0e3

In [ ]:
sampleQueryInput = {
    'iprcode': '0171670'
}  

In [ ]:
try:
  print(ProductDatabase.singleProductQuery({'iprcode':'12345'}))
except Exception as e:
  print(e)

key is not convertable to in 12345


In [ ]:
ProductDatabase.singleProductQuery(sampleQueryInput)

ValueError: key is not convertable to in 0171670

In [ ]:
#export
def lambdaSingleQuery(event, _):
  key, value = Event.from_dict(event).firstKey()
  try:
    result = ProductDatabase.singleProductQuery({key:value}).to_dict()
  except Exception as e:
    return Response.returnError(f'{e}')
  return Response.returnSuccess(body = result)

In [ ]:
event = Event(body = json.dumps(sampleQueryInput)).to_dict()
lambdaSingleQuery(event,'')

In [ ]:
result = lambdaSingleQuery(Event.getInput({'iprcode':'11234'}),'')
Response.parseBody(result)

### AllQuery

In [ ]:
from s3bz.s3bz import Requests
import zlib
%time url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key='allData-json.zl')
print(url)
# response = requests.get(url)
def retrieveJsonZl(url):
  response:requests.Response = requests.get(url)
  if response.status_code != 200:
    print(response.json())
  else:
    content = response.content
    jsonDb = zlib.decompress(content).decode()
    newDb = pd.read_json(jsonDb,orient='split')
    return newDb
%time newDb = retrieveJsonZl(url)
newDb.head()

In [ ]:
items = filter(lambda x: x != '', newDb['online_category_l1_en'])
len(list(newDb['online_category_l1_en'])) - len(list(items))

In [ ]:
items = filter(lambda x: x != '', newDb['online_category_l1_en'])
len(list(items))

In [ ]:
newDb.columns

In [ ]:
newDb['pr_suref3'].value_counts()

In [ ]:
#export
def lambdaAllQuery(event, *args):
  url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key='allData-json.zl')
  return Response.getReturn(body = {'url': url})

In [ ]:
from s3bz.s3bz import Requests
%time url = Response.fromDict(lambdaAllQuery('', '')).body['url']
%time newDb = retrieveJsonZl(url)
newDb.shape

#### AllQueryFeather

In [ ]:
#export
def lambdaAllQueryFeather(event, *args):
  key = 'allData'
  bucket = INVENTORY_BUCKET_NAME
  url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key=key)
  hashCode = pdUtils.loadRemoteHash(key=key, bucket=bucket, useUrl = True)
  return Response.getReturn(body = {'url': url, 'hash': hashCode})

In [ ]:
%time r = lambdaAllQueryFeather('')
%time result = Response.fromDict(r).body
url = result['url']
%time pd.read_feather(url)

In [ ]:
database = pd.read_feather(result['url'])
assert pdUtils.getDfHash(database) == result['hash'] , 'database and hash are not the same'

## Update s3, checking scan time

In [ ]:
%%time
print('checking scan time')
len(list(ProductDatabase.scan()))

In [ ]:
from nicHelper.schema import validateUrl

In [ ]:
input_ = {'cprcode': 123, 'iprcode': 123}
result = validateUrl('https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json', input_)

In [ ]:
result.cprcode
result.iprcode

123

In [ ]:
from nicHelper.dictUtil import genSchema
import yaml

In [ ]:
schema = genSchema(input_,format_='yaml')
print(schema)

$schema: http://json-schema.org/schema#
properties:
  cprcode:
    type: integer
  iprcode:
    type: integer
required:
- cprcode
- iprcode
type: object



In [ ]:
schemaDict = yaml.load(schema)

<ipython-input-17-e5d2d005a550>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  schemaDict = yaml.load(schema)


In [ ]:
import json
json.dumps(schemaDict)

'{"$schema": "http://json-schema.org/schema#", "properties": {"cprcode": {"type": "integer"}, "iprcode": {"type": "integer"}}, "required": ["cprcode", "iprcode"], "type": "object"}'

In [ ]:
import os
os.listdir('/tmp/')

['tmpFile',
 'input',
 'hsperfdata_role-agent',
 'tmpPath',
 'tmpw7pzt324.json',
 'testdict',
 'test2',
 'test4',
 'test',
 'tempFile.bz',
 'tempFile',
 'test1',
 'product.json',
 'teststr',
 '.ICE-unix',
 'feather',
 'fish.ec2-user',
 'zlibJsonCache.zl',
 'test3']

In [ ]:
import pandas as pd
pd.read_json('/tmp/product.json',orient = 'records')

ValueError: Unexpected character found when decoding object value

In [ ]:
with open ('/tmp/product.json', 'r') as f:
  productJson = json.load(f)

JSONDecodeError: Expecting ',' delimiter: line 1 column 6291457 (char 6291456)